In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))


import DifferentiationInterface.src.diffipy.diffipy as dp

2024-06-19 11:39:00.980866: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 11:39:00.980891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 11:39:00.981403: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-19 11:39:01.382116: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax', 'aadc'] 

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5
    N = 1000000
    seed = 1

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')
    r = dp.variable(r_value, 'input','r')
    sigma = dp.variable(sigma_value, 'input','sigma')
    dt = dp.variable(dt_value, 'input','dt')
    z = dp.variable(z_value, 'randomVariableNormal','z')

    func_input_variables = [s0, K, r, sigma, dt, z]
    diff_variables = [s0, K, r, sigma, dt]

    # Record Tape: Standard Monte Carlo

    s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
    payoff = dp.if_(s > K, s - K, 0)
    PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

    ### Test performance # seed and NewSample not working for aadc yet
    dp.seed(seed)
    pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(func_input_variables, diff_variables, warmup_iterations = 1, test_iterations = 3)


Backend              Eval-Result  mean runtime    variance runtime    gradient directions: {'K', 'dt', 'sigma', 'r', 's0'}
numpy                10.023520    0.085915        0.000017            [0.5002012963828406, -0.36360554602765655, 39.996882379789156, 39.89751398876251, 7.9844421986052785]
numpy_as_func        10.023520    0.072257        0.000000            {'s0': 0.5002013217847434, 'K': -0.36360553590242256, 'r': 39.99688241371757, 'sigma': 39.89751398751906, 'dt': 7.984442234842958}
torch                10.020878    0.007240        0.000000            [0.4992237389087677, -0.3627409040927887, 39.9015007019043, 39.90696334838867, 7.981120586395264]
torch_as_func        10.020878    0.001361        0.000000            [tensor(0.4992), tensor(-0.3627), tensor(39.9015), tensor(39.9070), tensor(7.9811)]
tensorflow           9.999600     0.031054        0.000023            [0.4993573, -0.36305574, 39.936134, 39.80617, 7.9677324]
tensorflow_as_func   9.999600     0.006897        0.000